In [56]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests
import math
import geopandas

In [57]:

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [58]:
response = requests.get('https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json')
json = response.json()
json.keys()

dict_keys(['@context', '@graph'])

In [59]:
df_flat_graph = pd.json_normalize(json_trabajando) 
df_flat_graph                                

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6070995,Jardines Gregorio Ordóñez,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28006,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,Jardines en memoria de Gregorio Ordóñez. Este ...,2,,,Jardines Gregorio Ordóñez
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6010361,Jardines San Francisco el Grande,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167,Este jardín se encuentra en el solar de 4.200 ...,2,,,Jardines San Francisco el Grande
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5964519,Jardines de Andrés Saborit,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28003,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970,Jardines situados en la calle Raimundo Fernánd...,2,,Zonas infantiles Zonas de mayores Circuito Bio...,Jardines de Andrés Saborit
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5896013,Jardines de El Buen Retiro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28001,PLAZA INDEPENDENCIA 7,40.419641,-3.687934,Jardín Histórico Artístico creado en el siglo ...,2,Horario del parque Primavera y verano (de a...,Zonas infantiles: Zona del Pinar (Puerta del ...,Jardines de El Buen Retiro
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,NaN,10984428,Jardines de Gloria Fuertes,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28016,CALLE MADRE DE DIOS 2,40.461518,-3.675118,Jardín dedicado a la poetisa y escritora de cu...,2,,Zona infantil,Jardines de Gloria Fuertes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6028228,Vivero Estufas de El Retiro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28009,GLORIETA ANGEL CAIDO 4 Parque del Retiro. Entr...,40.409899,-3.682331,Recinto construido en 1889 para albergar diver...,0,Invierno de 7 a 23 horas. El acceso al público...,,Vivero Estufas de El Retiro
199,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6028009,Vivero de la Casa de Campo,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28011,PASEO AZUL 2,40.419842,-3.728697,La primera noticia documental la tenemos en 18...,2,Invierno de 8 a 15 horas Verano de 7 a 14 hora...,,Vivero de la Casa de Campo
200,https://datos.madrid.es/egob/catalogo/tipo/ent...,NaN,6028007,Vivero municipal de Migas Calientes,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28040,AUTOVIA M-30 2200,40.445356,-3.739974,Centro de producción donde se realiza el culti...,0,De 8 a 15 horas.,,Vivero municipa

In [60]:
df_jardines = df_flat_graph[['title', 'address.street-address', 'location.latitude', 'location.longitude']]
df_jardines

,title,address.street-address,location.latitude,location.longitude
0,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595
1,Jardines San Francisco el Grande,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167
2,Jardines de Andrés Saborit,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970
3,Jardines de El Buen Retiro,PLAZA INDEPENDENCIA 7,40.419641,-3.687934
4,Jardines de Gloria Fuertes,CALLE MADRE DE DIOS 2,40.461518,-3.675118
...,...,...,...,...
198,Vivero Estufas de El Retiro,GLORIETA ANGEL CAIDO 4 Parque del Retiro. Entr...,40.409899,-3.682331
199,Vivero de la Casa de Campo,PASEO AZUL 2,40.419842,-3.728697
200,Vivero municipal de Migas Calientes,AUTOVIA M-30 2200,40.445356,-3.739974
201,Zona verde C/ Fresnedillas,CALLE FRESNEDILLAS 1 C/ Los Gavilanes y C/ Gab...,40.477264,-3.743003


In [72]:
df_park = df_jardines.rename({'title': 'Name_park', 'address.street-address': 'address_park', 'location.latitude': 'latitude_park', 'location.longitude': 'longitude_park'}, axis=1)
df_park

,Name_park,address_park,latitude_park,longitude_park
0,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595
1,Jardines San Francisco el Grande,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167
2,Jardines de Andrés Saborit,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970
3,Jardines de El Buen Retiro,PLAZA INDEPENDENCIA 7,40.419641,-3.687934
4,Jardines de Gloria Fuertes,CALLE MADRE DE DIOS 2,40.461518,-3.675118
...,...,...,...,...
198,Vivero Estufas de El Retiro,GLORIETA ANGEL CAIDO 4 Parque del Retiro. Entr...,40.409899,-3.682331
199,Vivero de la Casa de Campo,PASEO AZUL 2,40.419842,-3.728697
200,Vivero municipal de Migas Calientes,AUTOVIA M-30 2200,40.445356,-3.739974
201,Zona verde C/ Fresnedillas,CALLE FRESNEDILLAS 1 C/ Los Gavilanes y C/ Gab...,40.477264,-3.743003


In [73]:
##TABLA SQL

In [74]:
connectionDB = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
engineDB = create_engine(connectionDB)
inspector_db = inspect(engineDB)

In [75]:
inspector_db.get_table_names()

['bicimad_stations', 'bicipark_stations']

In [76]:
query_1 = '''
SELECT *
FROM bicimad_stations

'''
df_1 = pd.read_sql_query(query_1, engineDB)
df_1

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [77]:
df_1[["longitude","latitude"]]=df_1["geometry.coordinates"].str.split(",",expand=True)
df_1['longitude'] = df_1['longitude'].str.replace('[','',regex=True)
df_1["latitude"]=df_1['latitude'].str.replace(']','',regex=True)
df_1['latitude'] = pd.to_numeric(df_1['latitude'])
df_1['longitude'] = pd.to_numeric(df_1['longitude'])
df_1

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",-3.726930,40.443420
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",-3.727295,40.448332


In [78]:
df_bici = df_1[['name', 'address','latitude','longitude']]
df_bici

,name,address,latitude,longitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


In [79]:
df_bici_map = df_bici.rename({'name': 'Name to bici_map', 'address': 'address to station', 'latitude': 'latitude_bici', 'longitude': 'longitude_bici'}, axis=1)
df_bici_map

,Name to bici_map,address to station,latitude_bici,longitude_bici
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


In [80]:
#df_conexion_table = pd.concat([df_park,df_bici_map], axis=1)
#df_conexion_table

In [81]:
pk_conexion = df_park.assign(key=2).merge(df_bici_map.assign(key=2), how='left', on='key')
pk_conexion.head().head()


,Name_park,address_park,latitude_park,longitude_park,key,Name to bici_map,address to station,latitude_bici,longitude_bici
0,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587


def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [22]:
#def distance_meters(40.4167278, -3.7033387, 40.433462, -3.678595):
    rad = math.pi/180
    start = lat_finish - lat_start
    finish = long_finish - long_start
    R = 6372.795477598
    a = (math.sin(rad*start/2))**2 + math.cos(rad*lat_start)*math.cos(rad*lat_finish)*(math.sin(rad*finish/2))**2
    distance = 2*R*math.asin(math.sqrt(a))
    return distance

SyntaxError: invalid syntax (3574368213.py, line 1)

In [90]:
def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dtlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dtlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [91]:
haversine(40.433462, -3.678595,40.417214, -3.701834)

2.6716522851766844

In [92]:
pk_conexion

,Name_park,address_park,latitude_park,longitude_park,key,Name to bici_map,address to station,latitude_bici,longitude_bici
0,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,Jardines Gregorio Ordóñez,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,2,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...,...,...,...,...,...
53587,Área Forestal de Tres Cantos,CARRETERA M-607 2100,40.582363,-3.705431,2,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
53588,Área Forestal de Tres Cantos,CARRETERA M-607 2100,40.582363,-3.705431,2,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
53589,Área Forestal de Tres Cantos,CARRETERA M-607 2100,40.582363,-3.705431,2,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
53590,Área Forestal de Tres Cantos,CARRETERA M-607 2100,40.582363,-3.705431,2,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


In [89]:
pk_conexion['distance'] = pk_conexion.apply(lambda x: haversine(x['latitude_park', 'longitude_park'], x['latitude_bici', 'longitude_bici']),axis=1)
pk_conexion

KeyError: 'key of type tuple not found and not a MultiIndex'